In [12]:
%%capture
!pip install pickle5
!pip install -U sentence-transformers
import pickle5
import tensorflow.keras as keras
import tensorflow.keras.layers as layers
import numpy as np
import json
import psycopg2
import pickle
from sentence_transformers import SentenceTransformer


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [16]:
!cp /content/drive/MyDrive/UTMIST/Humerus/Dev\ Files/postgres_access.txt /content/postgres_access.txt
file = "/content/postgres_access.txt"
dataManager = DataManager(file)
game_results = dataManager.cleanGameData(dataManager.fetchGameData())

humerusdb humerusdecks UofT2020 ls-efb01dd3082301e5fe99a590e045a554867bb266.cwusutjvyntq.us-west-2.rds.amazonaws.com 5432
Successfully connected to database.
Saving games... Skipping all rows where a game was not played.
Saved game from row 18 to dictionary and appended dictionary to list.
Saved game from row 20 to dictionary and appended dictionary to list.
Saved game from row 10 to dictionary and appended dictionary to list.
Saved game from row 64 to dictionary and appended dictionary to list.
Saved game from row 47 to dictionary and appended dictionary to list.
Saved game from row 23 to dictionary and appended dictionary to list.
Saved game from row 15 to dictionary and appended dictionary to list.
Saved game from row 36 to dictionary and appended dictionary to list.
Saved game from row 49 to dictionary and appended dictionary to list.
Saved game from row 8 to dictionary and appended dictionary to list.
Saved game from row 42 to dictionary and appended dictionary to list.
Saved game

In [17]:
raw_play_strings = [i[0] for i in game_results]
# 1 if win, 0 if loss
y_data = [1 if i[1] else 0 for i in game_results]

In [13]:
!cp /content/drive/MyDrive/UTMIST/Humerus/Dev\ Files/training_set_data_69.pkl /content/training_set_data_69.pkl

with open('/content/training_set_data_69.pkl', 'rb') as f:
    embed_table = pickle5.load(f)

In [14]:
embed_dict = {}
for i, x in enumerate(zip(embed_table['sentence'], embed_table['Bert_embed_69'])):
    embed_dict[x[0]] = x[1]

In [4]:
'''
The pca class
input matrix should be m x 768 where m is the number of examples.
'''
class pca:
    def __init__(self, cov_path):
        with open(cov_path, 'rb') as f:
            cache = pickle.load(f)

        self.eigenvector = cache["eigenvectors"]
        self.eigenvalue = cache["eigenvalues"]
        self.mean = cache["mean"]
        self.sd = cache["std"]
        
    def reduce_kdim(self, X, k):
        X = (X - self.mean) / self.sd
        X_reduced = np.dot(X, self.eigenvector[:, :k])
        return X_reduced


In [18]:
# on the fly generation
s_model = SentenceTransformer('distilbert-base-nli-mean-tokens')
!cp /content/drive/MyDrive/UTMIST/Humerus/Dev\ Files/pca_cache_69.pkl /content/pca_cache_69.pkl
pcaifier = pca('/content/pca_cache_69.pkl')

100%|██████████| 245M/245M [00:12<00:00, 20.3MB/s]


In [19]:
X_train, Y_train = [], []
dictkeys = list(embed_dict.keys())
for i, x in enumerate(raw_play_strings):
    try:
        embed = embed_dict[x]
        X_train.append(embed)
        Y_train.append(y_data[i])
    except:
        embed = pcaifier.reduce_kdim(s_model.encode(x), 69)
        X_train.append(embed)
        Y_train.append(y_data[i])

X_train = np.array(X_train)
Y_train = np.array(Y_train)


In [22]:
pred_net = predictionNetwork.from_scratch(69)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 69)                4830      
_________________________________________________________________
dense_1 (Dense)              (None, 69)                4830      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 70        
Total params: 9,730
Trainable params: 9,730
Non-trainable params: 0
_________________________________________________________________


In [ ]:
pred_net.train_model(X_train, Y_train, 69)

In [21]:
def make_model(input_size):
    model = keras.Sequential()
    model.add(layers.Dense(input_size, input_shape=(input_size, ), activation='relu'))
    model.add(layers.Dense(input_size))
    model.add(layers.Dense(1))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [6]:
class predictionNetwork:
    """basic binary classifier prediction network for CAH
    """
    def __init__(self, model): 
        self.model = model

    @classmethod
    def from_scratch(cls, input_shape):
        mdl=make_model(input_shape)
        mdl.summary()
        return cls(mdl)

    @classmethod
    def from_file(cls, input_shape, weight_path): 
        # get input shape
        # pass relative path strings
        mdl = make_model(input_shape)
        mdl.load_weights(weight_path)
        return cls(mdl)


    def save_models(self, folder, name):
        # assuming path: ./mdls/l & ./mdls/r for left & right models, respectively. And left and right are of same num.
        # naming convention: {mdl_type}_num.h5
        paths = os.listdir(folder)
        try:
            paths.remove('.ipynb_checkpoint')
        except:
            pass
        if len(paths) == 0:
            n = 1
        else:
            n = max([int(re.findall("\d+", i)[0]) for i in paths]) + 1
        self.model.save(name+'_{n}.h5'.format(n=str(n)))

    def train_model(self, x_train, y_train, n_epochs):
        print("-------STARTING TRAINING------")

        print("-----SHAPES-------")
        print("X:", x_train.shape)
        print("Y:", y_train.shape)

        self.model.fit(x=x_train, y=y_train, \
            epochs=n_epochs, verbose=1, \
            validation_split = 0.1, shuffle=True ) 
        
    def evaluate(self, x):
        # x can be single item or list!
        return self.model.predict(np.expand_dims(x, axis=0), batch_size=1)[0]



In [7]:
# A class to fetch and clean all game data from the Postgres DB for training 
class DataManager:

    # initialize DataManager class with text file containing postgres DB access info
    def __init__(self, file):
        self.file = file
        self.parsed_data = None
    # Returns a list of dictionaries containing each game's lobby data in JSON format

    def fetch(self):
        """
        fills self.parsed_data
        """
        self.parsed_data = self.cleanGameData(self.fetchGameData())

    def toJson(self, filename):
        if self.parsed_data is None:
            self.fetch()
        json.dump(self.parsed_data, open(filename, 'w'))

    def fetchGameData(self):

        # Access Postgres DB credentials
        f = open(file, "r")

        # Connect to Postgres DB
        DB_NAME = (f.readline()).strip()
        DB_USER = (f.readline()).strip()
        DB_PASS = (f.readline()).strip()
        DB_HOST = (f.readline()).strip()
        DB_PORT = (f.readline()).strip()

        print(DB_NAME, DB_USER, DB_PASS, DB_HOST, DB_PORT)

        conn = psycopg2.connect(database = DB_NAME, user = DB_USER, password = DB_PASS, 
                                        host = DB_HOST, port = DB_PORT)

        print("Successfully connected to database.")

        cur = conn.cursor()

        cur.execute("SELECT ID, LOBBY FROM massivedecks.lobbies")

        allGameData = []
        rows = cur.fetchall()
        
        print("Saving games... Skipping all rows where a game was not played.")

        for data in rows:
            rawGameData = data[1]

            # Skip all rows where a game was not played and the game history is not empty
            if "game" in rawGameData:
                if rawGameData["game"]["history"]:
                    allGameData.append(rawGameData.copy())
                    print("Saved game from row " + str(data[0]) + " to dictionary and appended dictionary to list.") 

        print("All game data selected from database and stored in list of dictionaries.")
        conn.close()

        return allGameData

    # Accepts list of dictionaries as input and returns list of tuples (call, play, result) as output
    def cleanGameData(self, data):
        flatten = lambda t: [item for sublist in t for item in sublist]

        results = []

        # Loop through every game
        for game in data:

            # Loop through every round in the game
            for gameRound in game["game"]["history"]:
                
                # Declare call card, to be added to tuple
                call = gameRound["call"]["parts"]

                winner = int(gameRound["winner"])

                # Loop through every player in the round
                for player in gameRound["plays"]:

                    # Declare play card and play result, to be added to tuple
                    plays = []
                    won = False

                    # Set result as won if player ID matches winner ID
                    if int(player) == winner:
                        won = True

                    # Loop through every play made by the player
                    for play in gameRound["plays"][player]["play"]:
                        played = play["text"]
                        plays.append(played)

                    # combine call and plays
                    # result = (call, plays, won)

                    call = flatten(call)
                    while _find_first_instance_by_type(call, dict) != -1 and len(plays): # make sure plays is not empty
                        call[_find_first_instance_by_type(call, dict)] = plays.pop(0)
                    
                    while _find_first_instance_by_type(call, dict) != -1: 
                        # if there is still styling in dict format in call, remove it
                        call[_find_first_instance_by_type(call, dict)] = "" 
                        
                    sentence = "".join(call)

                    results.append((sentence, won))

        print("Successfully cleaned all game data and returned it in a list of tuples.")
        return results


def _find_first_instance_by_type(list, t):
    """
    returns index of first instance of type t, -1 otherwise
    list: list,
    t: type to look for 
    """
    for ind, itm in enumerate(list):
        if type(itm) == t:
            return ind
    return -1





In [8]:
%%capture
# ghetto flask API
!pip install flask-ngrok
!pip install flask==0.12.2 # for compatability with flask-ngrok
from flask import Flask, request, jsonify
from flask_ngrok import run_with_ngrok


In [23]:
app = Flask(__name__)
app.config['JSONIFY_PRETTYPRINT_REGULAR'] = False
rd = 0
@app.route('/', methods=['GET', 'POST']) # (g)et_(p)robability
def eval_model():
    global embed_dict, s_model, pcaifier, pred_net, rd # change this later

    if request.method == 'POST':  #POST a json list only. for 1 element, do 1 elem list.
        raw_text = request.form.get('text')
        if type(raw_text) == str:
            raw_text = json.loads(raw_text.replace("'", '"'))

        try:
            embed = embed_dict[raw_text]
        except:
            embed = pcaifier.reduce_kdim(s_model.encode(raw_text), 69)
        
        payload = pred_net.evaluate(embed)
        return jsonify([str(i) for i in payload.flatten()])
        
    s = """<form method="POST">
                  Text: <input type="text" name="text"><br>
                  <input type="submit" value="Submit"><br>
              </form>"""
    return s

In [ ]:
payload = pred_net.evaluate(da)

In [ ]:


run_with_ngrok(app)
app.run()



 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://fbd2e9d6ff92.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [24/Jan/2021 18:02:01] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [24/Jan/2021 18:02:03] "GET /favicon.ico HTTP/1.1" 404 -
[2021-01-24 18:02:14,014] ERROR in app: Exception on / [POST]
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/flask/app.py", line 1982, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/lib/python3.6/dist-packages/flask/app.py", line 1614, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/lib/python3.6/dist-packages/flask/app.py", line 1517, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/usr/local/lib/python3.6/dist-packages/flask/_compat.py", line 33, in reraise
    raise value
  File "/usr/local/lib/python3.6/dist-packages/flask/app.py", line 1612, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/lib/python3.6/dist-packages/flask/app.py", line 1598, in dispatch_request
    return self.view_functions[rule.endpoint]